In [29]:
import requests
from multiprocessing.dummy import Pool as ThreadPool
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
def moneyfrom(in_str):
    get_data = [int(s) for s in 
                in_str.replace("-", " ").replace("–", " ").split(" ") if s.isdigit()]
    if len(get_data) == 2:
        if get_data[0] == 700:
            return 2
        if get_data[0] == 1500:
            return 3
    else:
        if get_data[0] == 700:
            return 1
        if get_data[0] == 2500:
            return 4

def getInfoFromResponse(data, resp, num_resp):
    if resp.status_code != 200:
        print("Error", num_resp)
        return
    if num_resp % 20 == 0:   
        print(resp, num_resp)
    bs = BeautifulSoup(resp.text, 'lxml')
    for item in bs.find_all('div', 'places_info'): 
        find_name = item.find('span', 'places_name')
        if find_name is not None: 
            name = find_name.text.strip()
        else: 
            name = None

        find_address = item.find('span', 'places_address')
        if find_address is not None:     
            address = find_address.text.strip()
        else: 
            address = None

        find_metro = item.find('span', 'places_metro')
        if find_metro is not None:     
            metro = find_metro.text.strip()[3:]
        else: 
            metro = None

        find_rating = item.find('div', 'rating_number')
        if find_rating is not None: 
            rating = item.find('div', 'rating_number').text.strip()
        else:
            rating = None
        
        find_money = item.find('span', "range s-tooltip")
        if find_money is not None:
            money = moneyfrom(find_money.get("data-title"))
        else:
            money = None

        data.append((name, address, metro, rating, money))

In [4]:
data = list()
FIRST_URL = "https://www.afisha.ru/msk/restaurants/restaurant_list/"
first_resp = requests.get(FIRST_URL)
getInfoFromResponse(data, first_resp, 1)

t = ThreadPool(2)
start_page = 2
end_page = 453
r = t.map(lambda x: getInfoFromResponse(data, 
                    requests.get("https://www.afisha.ru/msk/restaurants/restaurant_list/page%d/?view=list" % x), x), 
          range(start_page, end_page))
t.close()
t.join()

<Response [200]> 60
<Response [200]> 20
<Response [200]> 80
<Response [200]> 40
<Response [200]> 100
<Response [200]> 120
<Response [200]> 180
<Response [200]> 140
<Response [200]> 200
<Response [200]> 160
<Response [200]> 220
<Response [200]> 240
<Response [200]> 300
<Response [200]> 260
<Response [200]> 320
<Response [200]> 280
<Response [200]> 340
<Response [200]> 360
<Response [200]> 420
Error 428
Error 429
Error 430
Error 431
Error 432
Error 433
Error 434
Error 435
Error 436
Error 437
Error 438
Error 439
Error 440
Error 441
Error 442
Error 443
Error 444
Error 445
Error 446
Error 447
Error 448
Error 449
Error 450
Error 451
Error 452
<Response [200]> 380
<Response [200]> 400


In [ ]:
df = pd.DataFrame(np.array(data, dtype = object), columns=["Name", "Address", "Metro", "Rating", "Money"])

In [6]:
df.to_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

In [31]:
df = pd.DataFrame.from_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

In [32]:
df = df.sort_values('Rating', ascending=False).dropna()

In [33]:
df.shape

(1246, 5)

In [36]:
df.iloc[113]

Name                   Чито-Ра
Address         Казакова, 10/2
Metro      Курская, Чкаловская
Rating                     8.3
Money                        1
Name: 245, dtype: object

In [2]:
resp = requests.get("http://metro.mwmoskva.ru/stancii-spisok/po_rajonam/")

In [4]:
bs = BeautifulSoup(resp.text, 'lxml')

In [41]:
names = list()
for item in bs.find_all("div", "list-title"):
    names.append(item.text)

In [17]:
metro = list()
for list_item in bs.find_all("ul", "list-content"):
    metro_names = list()
    for metro_name in list_item.find_all("div", "list-name"):
        metro_names.append(metro_name.text.strip())
    metro.append(metro_names)

In [42]:
names

['в Академическом',
 'в Алексеевском',
 'на Арбате',
 'в районе Аэропорт',
 'в Бабушкинском',
 'в Басманном',
 'в Бескудниково',
 'в Бибирево',
 'в Богородском',
 'в Братеево',
 'в Северном Бутово',
 'в Южном Бутово',
 'в Бутырском',
 'в Войковском',
 'в Жулебино',
 'в Гагаринском',
 'в Головинском',
 'в Гольяново',
 'в Даниловском',
 'в Донском',
 'в Дорогомилово',
 'в Замоскворечье',
 'в Зюзино',
 'в Зябликово',
 'в Измайлово',
 'в Северном Измайлово',
 'в Коньково',
 'в Коптево',
 'в Красногорском',
 'в Красносельском',
 'в Крылатском',
 'в Кузьминках',
 'в Кунцево',
 'в Левобережном',
 'в Лефортово',
 'в Люблино',
 'в Марьиной Роще',
 'в Марьино',
 'в Северном Медведково',
 'в Мещанском',
 'в Митино',
 'в Московском районе',
 'в Нагатино-Садовники',
 'в Нагатинском Затоне',
 'в Нагорном',
 'в Нижегородском',
 'в Новогиреево',
 'в Новокосино',
 'в Обручевском',
 'в Северном Орехово-Борисово ',
 'в Южном Орехово-Борисово',
 'в Останкино',
 'в Отрадном',
 'в Перово',
 'в Печатнике',
 

In [28]:
metro

[['Академическая'],
 ['Алексеевская'],
 ['Александровский сад',
  'Арбатская',
  'Арбатская',
  'Боровицкая',
  'Смоленская',
  'Смоленская'],
 ['Аэропорт',
  'Динамо',
  'Петровский парк',
  'Петровский парк',
  'Сокол',
  'Петровский парк 2016 год'],
 ['Бабушкинская'],
 ['Бауманская',
  'Курская',
  'Курская',
  'Чистые пруды',
  'Чкаловская',
  'Рубцовская 2018 год'],
 ['Верхние Лихоборы', 'Окружная', 'Селигерская'],
 ['Алтуфьево', 'Бибирево'],
 ['Белокаменная', 'Бульвар Рокоссовского', 'Бульвар Рокоссовского'],
 ['Алма-Атинская', 'Борисово'],
 ['Бульвар Дмитрия Донского', 'Улица Старокачаловская'],
 ['Бульвар адмирала Ушакова',
  'Бунинская Аллея',
  'Улица Горчакова',
  'Улица Скобелевская'],
 ['Бутырская',
  'Савеловская',
  'Улица Милашенкова',
  'Фонвизинская',
  'Бутырская 2016 год',
  'Фонвизинская 2016 год'],
 ['Балтийская', 'Войковская'],
 ['Выхино',
  'Жулебино',
  'Котельники',
  'Лермонтовский проспект',
  'Косино 2018 год',
  'Юго-Восточная 2018 год'],
 ['Площадь Гагари

Разбираемся с районами

In [52]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [55]:
mo_gdf = gpd.read_file('Users/anton/Downloads/demography/demography.shp',  encoding="windows-1251")
mo_gdf = mo_gdf.drop(list(mo_gdf.columns[2:13]), axis=1)
mo_gdf.head()

OSError: no such file or directory: 'Users/anton/Downloads/demography/demography.shp'